In [9]:
import sqlite3
import pandas as pd
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import joblib

# Step 1: Read Data from SQLite3 Table
conn = sqlite3.connect('Cleaned_DB.db')
query = "SELECT ax, ay, az, gx, gy, gz, label FROM mockdata"
df = pd.read_sql_query(query, conn)
conn.close()

# Step 2: Preprocess the Data
X = df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values
y = df['label'].values

# Encode normal as 1 and anomalie as 0
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split the data into training and testing sets
X_trainn, X_testt, y_trainn, y_testt = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Step 3: Train the Model
knn_clf = KNeighborsTimeSeriesClassifier()
knn_clf.fit(X_trainn, y_trainn)

# Step 4: Evaluate the Model (Optional)
y_pred = knn_clf.predict(X_testt)
print("Accuracy:", accuracy_score(y_testt, y_pred))

joblib.dump(knn_clf, 'knn_model.joblib')

Accuracy: 0.7394781144781145


['knn_model.joblib']

In [20]:
import joblib
import pandas as pd

n_data = {
    'ax': [-0.304688],
    'ay': [0.424316],
    'az': [-0.908691],
    'gx': [-6.793893],
    'gy': [-0.725191],
    'gz': [0.618321]
}
a_data = {
    'ax': [0.851563],
    'ay': [0.544434],
    'az': [0.144287],
    'gx': [-6.763359],
    'gy': [-1.083969],
    'gz': [-2.129771]
}

loaded_knn_clf = joblib.load('knn_model.joblib')

n_df = pd.DataFrame(n_data)
a_df = pd.DataFrame(a_data)

# Predict using the loaded model
n_pred = loaded_knn_clf.predict(n_df.values)
a_pred = loaded_knn_clf.predict(a_df.values)

print("normal", n_pred)
print("anomalie", a_pred)

normal [1]
anomalie [0]


In [21]:
import joblib
import pandas as pd
import sqlite3

conn = sqlite3.connect('Cleaned_DB.db')
query_anomalie = "SELECT ax, ay, az, gx, gy, gz FROM mockdata WHERE label = 'anomalie' LIMIT 50"
df_anomalie = pd.read_sql_query(query_anomalie, conn)
conn.close()

loaded_knn_clf = joblib.load('knn_model.joblib')
anomalie_pred = loaded_knn_clf.predict(df_anomalie.values)
print(anomalie_pred)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0]
